In [1]:
using StatsBase
using Combinatorics

In [2]:
Z = [1 1 2 2 3 3 4 4 4 5 5] # group partition
D = [3 4 2 5 6 4 3 2 5 2 2] # degree sequence

1×11 Array{Int64,2}:
 3  4  2  5  6  4  3  2  5  2  2

First thing we'll do: let's check the formula

$$\sum_{R \in [n]^\ell} \mathbb{I}(\#(\mathbf{z}_R) = p) \sigma(\theta_R) = \sum_{y: \#y = p}\prod_{a = 1}^{\ell}v_{y_a}\;,$$

In this formula, we're allowing R to range over $[n]^\ell$, where $\ell$ is the number of nodes per hyperedge, and $n$ the number of nodes.  This is **different** from the current version of the nodes, and matches the conversation we had over email. 



In [3]:
# Naive evaluation of the sum (LHS)

function evalSum(p, Z, D)
    n = length(Z)
    ℓ = sum(p)

    S = 0

    T = Iterators.product((1:n for i = 1:ℓ)...)

    for R in T
        a = countmap(vec(Z[collect(R)]))
        a = -sort(-collect(values(a)))
        if a == p
            S += prod(D[collect(R)])
        end
    end
    return(S)
end

# Faster evaluation of the sum (RHS)

function evalSum2(p, Z, D)
    n = length(Z)
    ℓ = sum(p)
    S = 0

    V = [sum([(Z[i] == s)*D[i] for i = 1:n]) for s = 1:maximum(Z)] # vector of volumes

    P = Iterators.product((1:maximum(Z) for i = 1:ℓ)...)
    for p_ in P
        a = countmap(vec(collect(p_)))
        a = -sort(-collect(values(a)))
        if a == p
          S += prod(V[collect(p_)])
        end
    end

    return(S)
end

evalSum2 (generic function with 1 method)

In [17]:
# test: check that these two functions give the same result on all partitions

ℓ = 4

for i = 1:ℓ
    for j = 1:i
        for p in partitions(i,j)
            println(evalSum(p, Z, D) == evalSum2(p, Z, D))
        end
    end
end

true
true
true
true
true
true
true
true
true
true
true


Ok, looks good! Next thing we're going to do is try the currence relation from the notes.

In [18]:
function evalSums(Z, D, ℓ)
    
    V = [sum([D[i]*(Z[i] == j) for i in 1:length(Z)]) for j in 1:maximum(Z)]
    μ = [sum(V.^i) for i = 1:ℓ]

    function correctOvercounting(M, p)
        pk = p[end]
        S = 0
        for i = 1:length(p)-1
            p_ = copy(p)[1:(end-1)]
            p_[i] += pk
            S += M[-sort(-p_)]
        end
        return(S)
    end

    M = Dict()

    for i = 1:ℓ
        for j = 1:i # number of nonzero entries
            for p in partitions(i, j)
                pk = p[j]
                M[p] = μ[p[end]]*get(M, p[1:(end-1)], 1) - correctOvercounting(M,p)
            end
        end
    end
    N = Dict()
    for p in keys(M)
        factor = 1
        counts = values(countmap(p))
        for c in counts
            factor *= factorial(c) 
        end

        N[p] = M[p] * multinomial(p...)/factor
    end
    return(N)
end

evalSums (generic function with 2 methods)

In [21]:
ℓ = 6
M = evalSums(Z, D, ℓ)

Dict{Any,Any} with 29 entries:
  [2, 2, 1]          => 2.30508e7
  [3, 3]             => 5.32311e7
  [3, 1]             => 317768.0
  [6]                => 2.23939e6
  [1, 1, 1, 1]       => 346080.0
  [3, 2]             => 6.28862e6
  [4, 2]             => 8.44323e7
  [2, 2]             => 220614.0
  [3, 2, 1]          => 7.7669e8
  [3]                => 2750.0
  [1, 1]             => 1130.0
  [1, 1, 1]          => 24576.0
  [2]                => 314.0
  [2, 1, 1]          => 1.17562e6
  [2, 1]             => 27546.0
  [3, 1, 1, 1]       => 4.5397e8
  [2, 1, 1, 1, 1]    => 2.68128e8
  [4, 1]             => 3.58783e6
  [2, 2, 2]          => 1.77499e8
  [4]                => 25058.0
  [1]                => 38.0
  [5, 1]             => 4.00611e7
  [5]                => 234638.0
  [1, 1, 1, 1, 1, 1] => 0.0
  [2, 1, 1, 1]       => 2.69976e7
  ⋮                  => ⋮

In [24]:
ℓ = 5
for i = 1:ℓ
    for j = 1:i
        for p in partitions(i,j)
            println(evalSum2(p, Z, D) == M[vec(p)])
        end
    end
end

true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
